# UCL Python Data Working Group February 2021
## Basic Jupyter Notebook Use Walkthrough
## 1. Introduction to Jupyter

### UCL Computer Science Postgraduate Admissions Example Walkthrough

<img align="right" width="200" height="200" src="https://sammcilroy.github.io/images/cop.jpg?raw=true#right">

Jupyter is an development environment for working interactively with Python (and other languages) and is used heavily in data analysis work. It allows you to combine working Python code with text, images and visualisations making it ideal for exploratory data analysis and report writing/sharing. The combination of code and readable text also make Jupyter a useful tool for documentation of processes. We will use synthesized UCL admissions data to walk through the basic functionality of notebooks including data imports, cleaning and manipulaton, basic analysis and visualisation.

Code is run in individual 'cells' like this one. You can insert and rearrange cells in any order, run them individually or all at once. 

Double click on this cell to inspect it. You can see markdown/HTML text being used to create the titles, text and images in this cell. For more info on markdown a useful resource is https://www.markdowntutorial.com
    



## 2. Setup, Python Libraries

The first code cell in most Jupyter files will be expected to be your imported libraries or any other general setup for your work.

In [123]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

## 3 Working with Data, The Pandas Library

The Pandas python library is one of the most common tools for working with dataset in python. It allows for importing of data from various text formats, from databases/SQL queries and from APIs. Data can be manipulated, cleaned and analysed replicating the function of SQL or any typical spreadsheet software which you may be using in your own non-python processes.

## 3.1 Importing Data

We'll use the Pandas library to import data from CSVs containing synthesized UCL Portico (Student Records) Data on Admissions and Offers Made to UCL Postgraduate Computer Science courses.

In [124]:
'''
Stadard Python comments are also useful when 
a markdown cell would be overkill/cumbersome
'''
# Importing the data...

admissions = pd.read_csv('ucl_cs_pg_admissions_synthesized.csv')
course_codes = pd.read_csv('ucl_course_codes.csv')
cod_codes = pd.read_csv('ucl_country_of_domicile_codes.csv')

## 3.2 Inspecting and Manipulating Data
We can continue using Pandas to inspect our data and make any changes/cleaning necessary before moving on with any analysis...

In [125]:
# look at data, pandas head()
admissions.head()

,CAP_AYRC,CAP_CRSC,STU_GEND,STU_CODC,CAP_IDRC
0,2019,TMSCOMSSML01,M,5826,CFUF
1,2019,TMSCOMSSSE01,F,672,C
2,2020,TMSCOMSCGV01,M,764,R
3,2019,TMSCOMSSSE01,M,631,R
4,2019,TMSCOMSING01,M,771,R


In [126]:
course_codes.head()

,CRS_CODE,CRS_NAME
0,TAFEDISCLD01,Affil - Cat 2 Clinical Dentistry
1,TAFEDISORT01,Affil - Cat 1 Orthodontics
2,TAFEDISPDD01,Affil - Cat 1 Paediatric Dentistry
3,TAFEENSEEE01,Affil - Cat 1 Electronic & Electrical Engineering
4,TAFEENSMMO01,Affil - Cat 1 Microwaves and Modern Optics


In [127]:
cod_codes.head()

,COD_CODE,COD_NAME
0,110,East Riding
1,111,North East Lincolnshire
2,112,North Lincolnshire
3,113,York
4,114,Bournemouth


Inspecting the admissions data....

In [128]:
# look at the shape of a dataset (rows, columns)
admissions.shape

(12751, 5)

Seeing the shape of the imported data we can note we see we have 12751 applications made with 5 dimensions of data on these:

* CAP_AYRC: The year of the application made
* CAP_CRSC: The course (code) applied to
* STU_GEND: The gender of the applicant
* STU_CODC: The country of domicile (code) of the applicant
* CAP_IDRC: The offer made to the applicant and their response

In [129]:
# look at individual columns with indexing, check unique values
admissions['CAP_AYRC'].unique()

array([2019, 2020, 2018])

In [130]:
admissions['CAP_CRSC'].unique()

array(['TMSCOMSSML01', 'TMSCOMSSSE01', 'TMSCOMSCGV01', 'TMSCOMSING01',
       'TMSCOMSMCL01', 'TMSCOMSFRM01', 'TMSCOMSRAC01', 'TMSCOMSWEB01',
       'TMSCOMSCFI01', 'TMSCOMSINF01', 'TMSCOMSFSE01', 'TMSCOMSDDI19'],
      dtype=object)

In [131]:
len(admissions['CAP_CRSC'].unique())

12

We note that the data shows applications over 3 years for 12 courses. We'll continue using Pandas to manipulate/clean this admissions data so it is more useful for analysis and sharing...

In [132]:
# renaming columns, python dictionary
admissions.rename(columns={'CAP_AYRC': 'Year',
                          'CAP_CRSC': 'Course Code',
                          'STU_GEND': 'Gender',
                          'STU_CODC': 'Domicile Code',
                          'CAP_IDRC': 'Decision Response'}, inplace=True)

admissions.head()

,Year,Course Code,Gender,Domicile Code,Decision Response
0,2019,TMSCOMSSML01,M,5826,CFUF
1,2019,TMSCOMSSSE01,F,672,C
2,2020,TMSCOMSCGV01,M,764,R
3,2019,TMSCOMSSSE01,M,631,R
4,2019,TMSCOMSING01,M,771,R


In [133]:
# changing data types
'''
we'll want to make sure any codes that could be interpreted as ints are seen as strings
'''
admissions['Year'] = admissions['Year'].astype(str)
admissions['Course Code'] = admissions['Course Code'].astype(str)
admissions['Domicile Code'] = admissions['Domicile Code'].astype(str)


In [134]:
# lookups/joins 
'''
use the code lookup tables to replace the course and domicile codes with their actual values
'''
course_codes.rename(columns={"CRS_CODE": 'Course Code'}, inplace=True)
admissions = pd.merge(admissions, course_codes, on='Course Code', how='left')

In [135]:
cod_codes.rename(columns={'COD_CODE': 'Domicile Code'}, inplace=True)
admissions = pd.merge(admissions, cod_codes, on='Domicile Code', how='left')

admissions.head()

,Year,Course Code,Gender,Domicile Code,Decision Response,CRS_NAME,COD_NAME
0,2019,TMSCOMSSML01,M,5826,CFUF,MSc Computational Statistics and Machine Learning,England
1,2019,TMSCOMSSSE01,F,672,C,MSc Software Systems Engineering,India
2,2020,TMSCOMSCGV01,M,764,R,"MSc Computer Graphics, Vision and Imaging",United Arab Emirates
3,2019,TMSCOMSSSE01,M,631,R,MSc Software Systems Engineering,China
4,2019,TMSCOMSING01,M,771,R,MSc Computer Science,United States of America


In [136]:
# reordering columns
admissions = admissions[['Year',
                        'Course Code',
                         'Gender',
                        'CRS_NAME',
                        'Domicile Code',
                        'COD_NAME',
                        'Decision Response']]

admissions.rename(columns={'CRS_NAME': 'Course Name',
                          'COD_NAME': 'Domicile'}, inplace=True)

admissions.head()

,Year,Course Code,Gender,Course Name,Domicile Code,Domicile,Decision Response
0,2019,TMSCOMSSML01,M,MSc Computational Statistics and Machine Learning,5826,England,CFUF
1,2019,TMSCOMSSSE01,F,MSc Software Systems Engineering,672,India,C
2,2020,TMSCOMSCGV01,M,"MSc Computer Graphics, Vision and Imaging",764,United Arab Emirates,R
3,2019,TMSCOMSSSE01,M,MSc Software Systems Engineering,631,China,R
4,2019,TMSCOMSING01,M,MSc Computer Science,771,United States of America,R


In [137]:
# dropping unwanted columns
admissions.drop('Course Code', 1, inplace=True)
admissions.drop('Domicile Code', 1, inplace=True)
admissions.head()

,Year,Gender,Course Name,Domicile,Decision Response
0,2019,M,MSc Computational Statistics and Machine Learning,England,CFUF
1,2019,F,MSc Software Systems Engineering,India,C
2,2020,M,"MSc Computer Graphics, Vision and Imaging",United Arab Emirates,R
3,2019,M,MSc Software Systems Engineering,China,R
4,2019,M,MSc Computer Science,United States of America,R


In [138]:
# mapping one value to another
gender_map = {'M': 'Male', 'F': 'Female'}
admissions['Gender'] = admissions['Gender'].map(gender_map)
admissions.head()

,Year,Gender,Course Name,Domicile,Decision Response
0,2019,Male,MSc Computational Statistics and Machine Learning,England,CFUF
1,2019,Female,MSc Software Systems Engineering,India,C
2,2020,Male,"MSc Computer Graphics, Vision and Imaging",United Arab Emirates,R
3,2019,Male,MSc Software Systems Engineering,China,R
4,2019,Male,MSc Computer Science,United States of America,R


In [139]:
# sorting the data
admissions.sort_values('Year', ascending=False, inplace=True)

In [140]:
admissions.head()

,Year,Gender,Course Name,Domicile,Decision Response
6375,2020,Male,MSc Financial Risk Management,China,CFUF
6737,2020,Male,MSc Machine Learning,France,NaN
6679,2020,Male,MSc Computational Statistics and Machine Learning,China,R
6675,2020,Male,MSc Machine Learning,England,R
6672,2020,Male,MSc Computer Science,China,R


In [141]:
# applying custom functions

'''
define our own custom function to add meaning to the decision response codes
'''

def offer_made(decision):
    decision = str(decision)
    if decision.startswith('C'):
        return 'Conditional Offer'
    elif decision.startswith('U'):
        return 'Unconditional Offer'
    elif decision.startswith('R'):
        return 'Rejection'
    else:
        return 'None'
    
admissions['Decision Response'] = admissions['Decision Response'].apply(offer_made)
admissions.rename(columns={'Decision Response': 'Decision'}, inplace=True)
admissions.head()

,Year,Gender,Course Name,Domicile,Decision
6375,2020,Male,MSc Financial Risk Management,China,Conditional Offer
6737,2020,Male,MSc Machine Learning,France,None
6679,2020,Male,MSc Computational Statistics and Machine Learning,China,Rejection
6675,2020,Male,MSc Machine Learning,England,Rejection
6672,2020,Male,MSc Computer Science,China,Rejection


In [142]:
# saving the cleaned data to a new csv
admissions.to_csv('admissions_cleaned.csv', index=False)

## 4. Analysing the Data

Once the data is in a cleaned and ready state, we can continue by appying further Pandas/python functions to explore and analyse

In [143]:
'''
quick analysis by functions
'''
def offer(decision):
    decision = str(decision)
    if 'Conditional' in decision or 'Unconditional' in decision:
        return 1
    return 0

admissions['Offer'] = admissions['Decision'].apply(offer)
admissions.head()

,Year,Gender,Course Name,Domicile,Decision,Offer
6375,2020,Male,MSc Financial Risk Management,China,Conditional Offer,1
6737,2020,Male,MSc Machine Learning,France,None,0
6679,2020,Male,MSc Computational Statistics and Machine Learning,China,Rejection,0
6675,2020,Male,MSc Machine Learning,England,Rejection,0
6672,2020,Male,MSc Computer Science,China,Rejection,0


In [144]:
offers_made = int(admissions['Offer'].sum())
offers_made

2818

In [145]:
offers_made/len(admissions)

0.221002274331425

Applying aggregate/sum function to quickly count offers shows that 2818 offers were given which is an overall acceptace rate of 22.1%

In [146]:
# Segregating the data. Let's look at Msc Computer Science only
cs = admissions[admissions['Course Name'] == 'MSc Computer Science']
cs.head()

,Year,Gender,Course Name,Domicile,Decision,Offer
6672,2020,Male,MSc Computer Science,China,Rejection,0
6671,2020,Female,MSc Computer Science,China,Rejection,0
6664,2020,Female,MSc Computer Science,England,Rejection,0
6653,2020,Male,MSc Computer Science,England,Rejection,0
6620,2020,Female,MSc Computer Science,China,None,0


In [147]:
len(cs)

2389

In [148]:
offers_made = int(cs['Offer'].sum())

In [149]:
offers_made

534

In [150]:
offers_made/len(cs)

0.22352448723315194

MSc Computer Science has 534 offers and a similar acceptance rate of 22.3% over its 2389 applications the past 3 years

In [157]:
'''
Aggreating/Pivoting

Replicating pivot table functionality in Jupyter to quickly report on data and
see useful patterns
'''
offers_made_by_year_by_gender = pd.pivot_table(admissions, values='Offer', index=['Year', 'Course Name'], columns=['Gender'], aggfunc=np.sum)

In [158]:
offers_made_by_year_by_gender

Gender                                                  Female  Male
Year Course Name                                                    
2018 MSc Computational Finance                              21    56
     MSc Computational Statistics and Machine Learning      25    61
     MSc Computer Graphics, Vision and Imaging              17    27
     MSc Computer Science                                   50    99
     MSc Disability, Design and Innovation                   1     4
     MSc Financial Risk Management                          41   106
     MSc Financial Systems Engineering                       0     2
     MSc Information Security                               13    26
     MSc Machine Learning                                   53   102
     MSc Robotics and Computation                           14    38
     MSc Software Systems Engineering                       14    24
     MSc Web Science and Big Data Analytics                  5     8
2019 MSc Computational Finance                              19    55
     MSc Computational Statistics and Machine Learning      33    61
     MSc Computer Graphics, Vision and Imaging              12    18
     MSc Computer Science                                   43    89
     MSc Disability, Design and Innovation                   3     5
     MSc Financial Risk Management                          66   117
     MSc Financial Systems Engineering                       1     1
     MSc Information Security                               20    37
     MSc Machine Learning                                   49   109
     MSc Robotics and Computation                           12    31
     MSc Software Systems Engineering                       14    30
     MSc Web Science and Big Data Analytics                  3    11
2020 MSc Computational Finance                              50    68
     MSc Computational Statistics and Machine Learning      36    70
     MSc Computer Graphics, Vision and Imaging              24    34
     MSc Computer Science                                   91   162
     MSc Disability, Design and Innovation                   3     4
     MSc Financial Risk Management                          74   150
     MSc Financial Systems Engineering                       6     1
     MSc Information Security                               22    46
     MSc Machine Learning                                   68   133
     MSc Robotics and Computation                           23    35
     MSc Software Systems Engineering                       23    24
     MSc Web Science and Big Data Analytics                 10    15

,Offer
Offer,1.0
